# SageMaker Inference Recommender for Instant Recommendations

This notebook demonstrates how to get instant deployment recommendations from the DescribeModel API

The DescribeModel API can be found here: https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeModel.html

**This sample is provided for demonstration purposes, make sure to conduct appropriate testing if deriving this code for your own use-cases!**

## Set Up

In [ ]:
!pip install boto3 --upgrade

In [ ]:
import time
import boto3
import sagemaker
import pandas as pd
from uuid import uuid4
from datetime import datetime, timedelta

In [ ]:
sm_client = boto3.client(service_name="sagemaker")

## Step 1: Create the Model

For this sample notebook, we will use a simple model specifying only a Tensorflow DLC

In [ ]:
model_name = "instant-rec-demo-nb-model-" + str(uuid4())
print(model_name)

In [ ]:
region = boto3.Session().region_name
image = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="1.12.0",
    image_scope="inference",
    instance_type="ml.c5.4xlarge",
)
print(image)

In [ ]:
execution_role = sagemaker.get_execution_role()

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=execution_role, Containers=[{"Image": image}]
)

print(create_model_response)

## Step 2: Describe the Model

Describe the Model until `DeploymentRecommendations` goes into a `COMPLETED` state. On average, you should expect to have DeploymentRecommendations available within a second

In [ ]:
completed = timed_out = False

start = datetime.today()
max_duration = start + timedelta(minutes=1)

while not completed and not timed_out:
    describe_model_response = sm_client.describe_model(ModelName=model_name)
    deployment_recommendation = describe_model_response.get("DeploymentRecommendation")

    completed = (
        deployment_recommendation is not None
        and "COMPLETED" == deployment_recommendation.get("RecommendationStatus")
    )

    time.sleep(0.5)
    timed_out = datetime.today() > max_duration

    print(datetime.today(), "DeploymentRecommendation:", deployment_recommendation)

## Step 3: Visualize the DeploymentRecommendations

In [ ]:
pd.set_option("display.max_colwidth", None)
df = pd.DataFrame(
    deployment_recommendation.get("RealTimeInferenceRecommendations"),
    columns=["RecommendationId", "InstanceType", "Environment"],
)
display(df)

## Step 4: Cleanup

In [ ]:
sm_client.delete_model(ModelName=model_name)